## Coefficient matrix for different basis sets 

Evaluate the rotation of different basis sets through rotated water molecules

In [ ]:
from collections import defaultdict

import json

import numpy as np

import plotly.express as px

from ase.data import atomic_numbers
from ase.visualize import view

from pymatgen.core.structure import Molecule
from pymatgen.io.ase import AseAtomsAdaptor

import basis_set_exchange as bse

import torch

from e3nn import o3

from minimal_basis.predata.qchem import BaseQuantitiesQChem 
from minimal_basis.transforms.rotations import RotationMatrix

from instance_mongodb import instance_mongodb_sei

In [ ]:
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_water_calculations
basis_sets = collection.distinct('orig.rem.basis')
data = defaultdict(lambda: defaultdict(list))

for basis_set in basis_sets:
    basis_info = bse.get_basis(basis_set, fmt='json', elements=[1, 8])
    basis_info = json.loads(basis_info)

    electron_shells = {k: basis_info['elements'][k]['electron_shells'] for k in basis_info['elements'].keys()}
    angular_momenta = {k: [shell['angular_momentum'] for shell in electron_shells[k]] for k in electron_shells.keys()}
    angular_momenta = {k: [item for sublist in angular_momenta[k] for item in sublist] for k in angular_momenta.keys()}
    # Find tags with the basis set and sort by tags.idx
    find_tags = {'orig.rem.basis': basis_set}
    for doc in collection.find(find_tags).sort('tags.idx', 1):
        _alpha_coeff_matrix = doc['calcs_reversed'][0]['alpha_coeff_matrix']
        _alpha_eigenvalues = doc['calcs_reversed'][0]['alpha_eigenvalues']
        _alpha_fock_matrix = doc['calcs_reversed'][0]['alpha_fock_matrix']

        base_quantities_qchem = BaseQuantitiesQChem(
            fock_matrix=_alpha_fock_matrix,
            eigenvalues=_alpha_eigenvalues,
            coeff_matrix=_alpha_coeff_matrix,
        )

        _alpha_ortho_coeff_matrix = base_quantities_qchem.get_ortho_coeff_matrix()
        molecule = Molecule.from_dict(doc['orig']['molecule'])
        symbols = [site.specie.symbol for site in molecule.sites]
        atom_numbers = [atomic_numbers[symbol] for symbol in symbols]
        _basis_functions_orbital = []

        _irreps = ""
        for atom_number in atom_numbers:
            _angular_momenta = angular_momenta[str(atom_number)]
            _angular_momenta = np.array(_angular_momenta)
            _basis_functions = 2*_angular_momenta + 1
            for _basis_function in _basis_functions:
                if _basis_function == 1:
                    _basis_functions_orbital.extend(['s'])
                    _irreps += "+1x0e"
                elif _basis_function == 3:
                    _basis_functions_orbital.extend(['p', 'p', 'p'])
                    _irreps += "+1x1o"
                elif _basis_function == 5 and basis_set != '6-31g*':
                    _basis_functions_orbital.extend(['d', 'd', 'd', 'd', 'd'])
                    _irreps += "+1x2e"
                elif _basis_function == 5 and basis_set == '6-31g*':
                    _basis_functions_orbital.extend(['s', 'd', 'd', 'd', 'd', 'd'])
                    _irreps += "+1x0e+1x2e"

        _irreps = _irreps[1:]

        data[basis_set]['alpha_coeff_matrix'].append(_alpha_coeff_matrix)
        data[basis_set]['alpha_eigenvalues'].append(_alpha_eigenvalues)
        data[basis_set]['alpha_fock_matrix'].append(_alpha_fock_matrix)
        data[basis_set]['alpha_ortho_coeff_matrix'].append(_alpha_ortho_coeff_matrix)
        data[basis_set]['basis_functions_orbital'].append(_basis_functions_orbital)
        data[basis_set]['alpha_overlap_matrix'].append(base_quantities_qchem.overlap_matrix)
        data[basis_set]['irreps'].append(_irreps)
        data[basis_set]['euler_angles'].append(doc['tags']['euler_angles'])
        data[basis_set]['idx'].append(doc['tags']['idx'])
        data[basis_set]['molecule'].append(molecule)

for basis_set in basis_sets:
    for key in data[basis_set].keys():
        if key == 'molecule':
            continue
        data[basis_set][key] = np.array(data[basis_set][key])

In [ ]:
basis_set = "sto-3g"
quantity = "alpha_coeff_matrix"

## Structures

In [ ]:
all_molecules = data[basis_set]['molecule']
all_atoms = [AseAtomsAdaptor.get_atoms(molecule) for molecule in all_molecules]
view(all_atoms[0], viewer="x3d")

## $\alpha$ coefficient matrix

In [ ]:
fig = px.imshow(data[basis_set][quantity], animation_frame=0, range_color=[-1, 1])
labels = data[basis_set]['basis_functions_orbital'][0]
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Coefficient matrix from DFT calculation")
fig.show()


In [ ]:
new_coeff_matrix = []
calculated_coeff_matrix = data[basis_set][quantity]
original_coeff_matrix = calculated_coeff_matrix[0] 

for _idx, angles in enumerate(data[basis_set]['euler_angles']):

    torch_angles = torch.tensor(angles, dtype=torch.float64)

    irreps = data[basis_set]['irreps'][_idx]
    irreps = o3.Irreps(irreps)

    rotation_matrix = RotationMatrix(angle_type="euler", angles=angles)() 
    rotation_matrix = torch.tensor(rotation_matrix, dtype=torch.float64)
     
    D_matrix = irreps.D_from_matrix(rotation_matrix)
    D_matrix = D_matrix.detach().numpy()

    _new_coeff_matrix = np.zeros_like(original_coeff_matrix)
    for i in range(original_coeff_matrix.shape[1]):
        _new_coeff_matrix[:, i] = original_coeff_matrix[:, i] @ D_matrix.T

    new_coeff_matrix.append(_new_coeff_matrix)

new_coeff_matrix = np.array(new_coeff_matrix)

fig = px.imshow(new_coeff_matrix, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Rotated coefficient matrix")
fig.show()

In [ ]:
difference_matrices = np.abs(new_coeff_matrix) - np.abs(calculated_coeff_matrix)
fig = px.imshow(difference_matrices, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Difference between rotated and calculated coefficient matrix")
fig.show()

## Testing _d_-orbital transformations

In [ ]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [ ]:
D_matrices = []
for idx, angles in enumerate(data[basis_set]['angles']):
    _test_irreps = o3.Irreps('1x2e')
    if idx == 0:
        positions_0 = data[basis_set]['positions'][idx]
    positions = data[basis_set]['positions'][idx]

    if np.allclose(positions_0, positions):
        rotation_matrix = np.eye(3)
    else:
        _positions_0 = np.roll(positions_0, -1, axis=0)
        positions = np.roll(positions, -1, axis=0)
        rotation_matrix = rotation_matrix_from_vectors(_positions_0, positions)
    rotation_matrix = torch.tensor(rotation_matrix, dtype=torch.float64)
     
    D_matrix = _test_irreps.D_from_matrix(rotation_matrix)
    D_matrix = D_matrix.detach().numpy()
    D_matrices.append(D_matrix)

D_matrices = np.array(D_matrices)
fig = px.imshow(D_matrices, animation_frame=0, range_color=[-1, 1])
fig.show()

In [ ]:
d_indices = np.where(labels == 'd')[0]
data_to_store = []
for idx, angles in enumerate(data[basis_set]['angles']):
    orig_c = data[basis_set][quantity][0][d_indices, -1]
    rotated_c = data[basis_set][quantity][idx][d_indices, -1]
    orig_c = orig_c.reshape(-1, 1)
    rotated_c = rotated_c.reshape(-1, 1)
    D_matrix = D_matrices[idx]
    rotated = D_matrix @ orig_c 
    data_to_store.append(np.hstack((rotated, rotated_c)))


    if idx == 7:
        print('---')
        print('Rotated')
        print(rotated)
        print("Expected")
        print(rotated_c)


data_to_store = np.array(data_to_store)
fig = px.imshow(data_to_store, animation_frame=0, range_color=[-1, 1])
fig.show()